In [ ]:
# Phase 1

# 1.1

## given diabetic_data.csv file into a Pandas Dataframe
import pandas as pd
import numpy as np

df = pd.read_csv('data_files/diabetic_data.csv')

## Initial check

df.head()
df.info()
df.describe()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [ ]:
#1.2

##  process the dataframe to interpret ? as a standard NumPy NaN value.
import pandas as pd
df = pd.read_csv('data_files/diabetic_data.csv', na_values='?')

if df['weight'].isnull().mean() > 0.90:
    df.drop(columns=['weight'], inplace=True)
    print("Dropped")
    

Dropped


C:\Users\Dell\AppData\Local\Temp\ipykernel_26504\2099501224.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_files/diabetic_data.csv', na_values='?')


In [ ]:
#1.3

# read IDS_mapping file
mapping_df = pd.read_csv('data_files/IDs_mapping.csv' ,skiprows=10, nrows=31)

# marge df and mapping_df file
merged_df = pd.merge(df, mapping_df, on='discharge_disposition_id', how='left')

#find indexs for expired
expired_index = merged_df[
    merged_df['description'].str.contains('Expired', case=False, na=False)
].index

#before drop check df length
before = len(df)
#drop have expired rows and save df
df = merged_df.drop(expired_index)

#check new length of df
after = len(df)

print("Before:", before)
print("After:", after)
#check removed rows
print("Removed:", before - after)


Before: 101766
After: 100114
Removed: 1652


In [ ]:

#1.4

#Deduplication Strategy

#check for exact duplicate rows and count total number
duplicate_count = df.duplicated().sum()
print(duplicate_count)

#before drop check df length
before_drop = len(df)
#Remove the exact duplicate rows and assign the cleaned DataFrame back to the variable 'df'.
df = df.drop_duplicates(inplace=True)
##before drop check df length
after_drop = len(df)
#check removed rows
print(before_drop - after_drop)

In [ ]:
# Phase 2

# 2.1

import pandas as pd

# read diabetic_data.csv file
df = pd.read_csv('data_files/diabetic_data.csv')

# count diag_1 codes and select max 20
top_20_codes = df['diag_1'].value_counts().head(20)
print(top_20_codes)

diag_1
428      6862
414      6581
786      4016
410      3614
486      3508
427      2766
491      2275
715      2151
682      2042
434      2028
780      2019
996      1967
276      1889
38       1688
250.8    1680
599      1595
584      1520
V57      1207
250.6    1183
518      1115
Name: count, dtype: int64


In [ ]:
#2.2

import requests
from bs4 import BeautifulSoup
import time

#defined the function for web scraping
def get_icd9_details(code):

    #base url of the icd9 website
    url = 'http://icd9.chrisendres.com/index.php'

    # parameters that the website's search engine expects
    params = {
        'srchtype': 'diseases',
        'srchtext': str(code),
        'action' : 'search'
    }

    #send get request to url within paramters
    response = requests.get(url , params=params)

    #Check if the server responded successfully
    if response.status_code == 200:

        # Converts the raw HTML text into a structured soup object
        soup = BeautifulSoup(response.text, 'html.parser')

        #find <div> tag has the class name
        result = soup.find('div', class_= 'dlvl')

        if result:
            print(f'Code{code} fetched')
            #If the <div> was found, extract and return its text content
            return result.text.strip()
        else:
            #can't found to div element, return this
            return "Description Not Found"
    else:
        # If the website fails to load
        print(f'Failed code {code}')
        return "Fetch Failed"    

#A dictionary to store the results, it's used to both of code and description to store 
icd9_mapping = {}

## Loop through Top 20 codes identified 
for code in top_20_codes.index:
    # Call the function to scrape the description for the current code
    description = get_icd9_details(code)

    ## Store the result in our dictionary
    icd9_mapping[code] = description

    ## Print the final result for each code
    print(f'{code} -> {description}')

    ## Wait for 1 second before the next request to avoid getting blocked
    time.sleep(1)



Code428 fetched
428 -> 428 Heart failure
Code414 fetched
414 -> 414 Other forms of chronic ischemic heart disease
Code786 fetched
786 -> 786 Symptoms involving respiratory system and other chest symptoms
Code410 fetched
410 -> 410 Acute myocardial infarction
Code486 fetched
486 -> 486 Pneumonia, organism unspecified
Code427 fetched
427 -> 427 Cardiac dysrhythmias
Code491 fetched
491 -> 491 Chronic bronchitis
Code715 fetched
715 -> 715 Osteoarthrosis and allied disorders
Code682 fetched
682 -> 682 Other cellulitis and abscess
Code434 fetched
434 -> 434 Occlusion of cerebral arteries
Code780 fetched
780 -> 780 General symptoms
Code996 fetched
996 -> 996 Complications peculiar to certain specified procedures
Code276 fetched
276 -> 276 Disorders of fluid, electrolyte, and acid-base balance
38 -> Description Not Found
Code250.8 fetched
250.8 -> 250.8 Diabetes with other specified manifestations
Code599 fetched
599 -> 599 Other disorders of urethra and urinary tract
Code584 fetched
584 -> 58

In [ ]:
#2.3

# map icd9_mapping for new column Primary_Diagnosis_Desc
df['Primary_Diagnosis_Desc'] = df['diag_1'].map(icd9_mapping)

# outside the top 20, label them as "Other" using fillna
df['Primary_Diagnosis_Desc'] = df['Primary_Diagnosis_Desc'].fillna('Other')

#print first 10 rows to see the result
print(df[['diag_1' , 'Primary_Diagnosis_Desc']].head(10))

   diag_1                             Primary_Diagnosis_Desc
0  250.83                                              Other
1     276  276 Disorders of fluid, electrolyte, and acid-...
2     648                                              Other
3       8                                              Other
4     197                                              Other
5     414  414 Other forms of chronic ischemic heart disease
6     414  414 Other forms of chronic ischemic heart disease
7     428                                  428 Heart failure
8     398                                              Other
9     434                 434 Occlusion of cerebral arteries
